In [11]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import chevron
from bids import BIDSLayout
from bids.tests import get_test_data_path
import os

import nibabel as nib

from parameters import describe_image_size

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
dataset = "synthetic"

# Here we're using an example BIDS dataset that's bundled with the pybids tests
data_path = os.path.join(get_test_data_path(), dataset)

# Load the BIDS dataset
layout = BIDSLayout(data_path)

In [14]:
bf = layout.get(subject="01", extension="nii.gz", suffix="bold")[0]
metadata = bf.get_metadata()
filename = bf.path

img = nib.load(filename)
image_size = describe_image_size(img)

# join dictionaries
info = {**metadata, **image_size}

In [16]:
with open("partials/func.mustache", "r") as f:
    foo = chevron.render(f, info)

print(foo)

For the N-Back task,  run(s) of     fMRI data were collected.

The acquisiton parameters were:  slices acquired in a  fashion; repetition time,
TR= 2.5 s; echo time, TE=  ms; flip angle, FA=  deg; field of view,
FOV= 2x2x2 mm; matrix size= 64x64; voxel size= 128x128 mm; multiband factor= ;
in-plane acceleration factor= .

Each run was  minutes in length, during which  functional volumes were acquired.



In [18]:
default_func = {
    "TaskName": "TaskName",
    "nb_runs": "nb_runs",
    "suffix": "suffix",
    "PulseSequenceType": "PulseSequenceType",
    "ScanningSequence": "ScanningSequence",
    "SequenceVariant": "SequenceVariant",
    "nb_slices": "nb_slices",
    "so_st": "so_st",
    "RepetitionTime": "RepetitionTime",
    "echo_time": "echo_time",
    "FlipAngle": "FlipAngle",
    "fov": "fov",
    "mat_siz": "mat_siz",
    "vox_siz": "vox_siz",
    "mb_st": "mb_st",
    "pr_st": "pr_st",
    "length": "length",
    "nb_vol": "nb_vol",
}

# join dictionaries
info = {**default_func, **metadata}
info = {**info, **image_size}

with open("partials/func.mustache", "r") as f:
    foo = chevron.render(f, info)

print(foo)

For the N-Back task, nb_runs run(s) of suffix PulseSequenceType ScanningSequence SequenceVariant fMRI data were collected.

The acquisiton parameters were: nb_slices slices acquired in a  fashion; repetition time,
TR= 2.5 s; echo time, TE= echo_time ms; flip angle, FA= FlipAngle deg; field of view,
FOV= 2x2x2 mm; matrix size= 64x64; voxel size= 128x128 mm; multiband factor= ;
in-plane acceleration factor= .

Each run was length minutes in length, during which  functional volumes were acquired.

